In [1]:
# code from https://medium.com/district-data-labs/building-a-classifier-from-census-data-18f996c4d7cf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb

import mlflow
import mlflow.xgboost
import mlflow.sklearn
import pandas as pd

# prepare train and test data
names = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income',
]
data = pd.read_csv('census_data/adult.data', names=names)

import json
meta = {
    'target_names': list(data.income.unique()),
    'feature_names': list(data.columns),
    'categorical_features': {
        column: list(data[column].unique())
        for column in data.columns
        if data[column].dtype == 'object'
    },
}
with open('census_data/meta.json', 'w') as f:
    json.dump(meta, f, indent=2)

In [2]:
from sklearn.datasets.base import Bunch
from sklearn.preprocessing import LabelEncoder
def load_data():
    # Load the meta data from the file
    with open('census_data/meta.json', 'r') as f:
        meta = json.load(f)
    names = meta['feature_names']
    # Load the training and test data, skipping the bad row in the test data
    train = pd.read_csv('census_data/adult.data', names=names)
    test  = pd.read_csv('census_data/adult.test', names=names, skiprows=1)
    # Remove the target from the categorical features
    meta['categorical_features'].pop('income')
    # Return the bunch with the appropriate data chunked apart
    return Bunch(
        data = train[names[:-1]],
        target = train[names[-1]],
        data_test = test[names[:-1]],
        target_test = test[names[-1]],
        target_names = meta['target_names'],
        feature_names = meta['feature_names'],
        categorical_features = meta['categorical_features'],
        DESCR = "descr",
    )
dataset = load_data()

from sklearn.base import BaseEstimator, TransformerMixin
class EncodeCategorical(BaseEstimator, TransformerMixin):
    """
    Encodes a specified list of columns or all columns if None.
    """
    def __init__(self, columns=None):
        self.columns  = columns
        self.encoders = None
    def fit(self, data, target=None):
        """
        Expects a data frame with named columns to encode.
        """
        # Encode all columns if columns is None
        if self.columns is None:
            self.columns = data.columns
        # Fit a label encoder for each column in the data frame
        self.encoders = {
            column: LabelEncoder().fit(data[column])
            for column in self.columns
        }
        return self
    def transform(self, data):
        """
        Uses the encoders to transform a data frame.
        """
        output = data.copy()
        for column, encoder in self.encoders.items():
            output[column] = encoder.transform(data[column])
        return output
encoder = EncodeCategorical(dataset.categorical_features.keys())
dataset.data = encoder.fit_transform(dataset.data)
dataset.data_test = encoder.fit_transform(dataset.data_test)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
from sklearn.impute import SimpleImputer
import numpy as np

class ImputeCategorical(BaseEstimator, TransformerMixin):
    """
    Encodes a specified list of columns or all columns if None.
    """
    def __init__(self, columns=None):
        self.columns = columns
        self.imputer = None
    def fit(self, data, target=None):
        """
        Expects a data frame with named columns to impute.
        """
        # Encode all columns if columns is None
        if self.columns is None:
            self.columns = data.columns
        # Fit an imputer for each column in the data frame
        self.imputer = SimpleImputer(missing_values=0, strategy='most_frequent')
        self.imputer.fit(data[self.columns])
        return self
    def transform(self, data):
        """
        Uses the encoders to transform a data frame.
        """
        output = data.copy()
        output[self.columns] = self.imputer.transform(output[self.columns])
        return output
imputer = ImputeCategorical(['workclass', 'native-country', 'occupation'])
dataset.data = imputer.fit_transform(dataset.data)
dataset.data_test = imputer.fit_transform(dataset.data_test)

In [4]:
X_train = dataset.data
yencode = LabelEncoder().fit(dataset.target)
y_train = yencode.transform(dataset.target)

X_test = dataset.data_test
y_test = yencode.transform([y.rstrip(".") for y in dataset.target_test])
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# enable auto logging
mlflow.xgboost.autolog()

def grid_test_xgboost(colsample_tree, subsample, max_depth, min_child_weight, eta):
    with mlflow.start_run():
        # train model
        params = {
            'objective': 'multi:softprob',
            'num_class': 2,
            'eval_metric': 'mlogloss',
            'max_depth': max_depth,
            'min_child_weight': min_child_weight,
            'eta':eta,
            'subsample': subsample,
            'colsample_bytree': colsample_tree
        }
        model = xgb.train(params, dtrain, evals=[(dtrain, 'train')])

        # evaluate model
        y_proba = model.predict(dtest)
        y_pred = y_proba.argmax(axis=1)
        loss = log_loss(y_test, y_proba)
        acc = accuracy_score(y_test, y_pred)

        # log metrics
        mlflow.log_metrics({'log_loss': loss, 'accuracy': acc})
        return acc
    
from sklearn.svm import SVC
def grid_test_svm(kernel, gamma, C):
    with mlflow.start_run():
        clf = SVC(kernel=kernel, gamma=gamma, C=C).fit(X_train, y_train)
        accuracy = clf.score(X_test, y_test)
        mlflow.log_metrics({'accuracy': accuracy})
        mlflow.log_params({'kernel': kernel, 'gamma': gamma, 'C': C})
        return accuracy

# Grid Search XGBoost and Sklearn

In [5]:
#values from https://blog.cambridgespark.com/hyperparameter-tuning-in-xgboost-4ff9100a3b2f
colsample_tree = [0.2, 0.5, 1.0]
subsample = [0.05, 1.0]
max_depth = [1, 5, 10]
min_child_weight = [1, 6, 10, 15]
eta = [.9, .3, .01, .005]

for i in colsample_tree:
    for j in subsample:
        for k in max_depth:
            for l in min_child_weight:
                for m in eta:
                    grid_test_xgboost(i, j, k, l, m)

kernel = ['rbf']
gamma = [1e-3, 1e-4]
C = [1, 10, 100, 1000]
for i in kernel:
    for j in gamma:
        for k in C:
            grid_test_svm(i, j, k)

[0]	train-mlogloss:0.50964
[1]	train-mlogloss:0.45950
[2]	train-mlogloss:0.42490
[3]	train-mlogloss:0.41392
[4]	train-mlogloss:0.39993
[5]	train-mlogloss:0.39182
[6]	train-mlogloss:0.37690
[7]	train-mlogloss:0.36655
[8]	train-mlogloss:0.36062
[9]	train-mlogloss:0.35788


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mlflow/xgboost.py:333: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  all_arg_names = inspect.getargspec(original)[0]  # pylint: disable=W1505


[0]	train-mlogloss:0.60618
[1]	train-mlogloss:0.55536
[2]	train-mlogloss:0.51369
[3]	train-mlogloss:0.49838
[4]	train-mlogloss:0.48000
[5]	train-mlogloss:0.46668
[6]	train-mlogloss:0.45264
[7]	train-mlogloss:0.43993
[8]	train-mlogloss:0.43135
[9]	train-mlogloss:0.42742


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mlflow/xgboost.py:333: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  all_arg_names = inspect.getargspec(original)[0]  # pylint: disable=W1505


[0]	train-mlogloss:0.68980
[1]	train-mlogloss:0.68661
[2]	train-mlogloss:0.68301
[3]	train-mlogloss:0.68032
[4]	train-mlogloss:0.67727
[5]	train-mlogloss:0.67414
[6]	train-mlogloss:0.67098
[7]	train-mlogloss:0.66778
[8]	train-mlogloss:0.66488
[9]	train-mlogloss:0.66239


...

[4]	train-mlogloss:0.32960
[5]	train-mlogloss:0.31201
[6]	train-mlogloss:0.29812
[7]	train-mlogloss:0.28864
[8]	train-mlogloss:0.28096
[9]	train-mlogloss:0.27523
[0]	train-mlogloss:0.68697
[1]	train-mlogloss:0.68092
[2]	train-mlogloss:0.67499
[3]	train-mlogloss:0.66917


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mlflow/xgboost.py:333: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  all_arg_names = inspect.getargspec(original)[0]  # pylint: disable=W1505


[4]	train-mlogloss:0.66347
[5]	train-mlogloss:0.65787
[6]	train-mlogloss:0.65238
[7]	train-mlogloss:0.64699
[8]	train-mlogloss:0.64170
[9]	train-mlogloss:0.63652
[0]	train-mlogloss:0.69005
[1]	train-mlogloss:0.68699
[2]	train-mlogloss:0.68395
[3]	train-mlogloss:0.68095


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mlflow/xgboost.py:333: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  all_arg_names = inspect.getargspec(original)[0]  # pylint: disable=W1505


[4]	train-mlogloss:0.67798
[5]	train-mlogloss:0.67503
[6]	train-mlogloss:0.67212
[7]	train-mlogloss:0.66923
[8]	train-mlogloss:0.66637
[9]	train-mlogloss:0.66354


In [89]:
import os

dirs = os.listdir("mlruns/0")
dictionary = []
for i in dirs:
    row = {}
    if "meta" in i or "DS_Store" in i:
        continue
        
    accuracy = float(open('mlruns/0/' + i + '/metrics/accuracy').read().split(" ")[1])
    row['accuracy'] = accuracy
    params_files = os.listdir('mlruns/0/' + i + '/params')
    model_params = {}
    for j in params_files:
        if 'kernel' in j:
            continue
        model_params[j] = open('mlruns/0/' + i + '/params/' + j).read()
    
    row['model_params'] = model_params
    if len(params_files) == 3:
        row['model'] = 'SVM'
    else:
        row['model'] = 'xgboost'
    dictionary.append(row)
    
df = pd.DataFrame(dictionary)
df.to_csv('mlflow_workflows.csv')